# 식시 - 양재역으로 검색
- 명칭, 주요 메뉴, 평점, 전화번호, 주소

In [1]:
import requests
import pandas as pd
from urllib.parse import quote

## 데이터 가져오기

In [10]:
base_url = 'https://www.siksinhot.com'
url = f'{base_url}/search?keywords=%EC%96%91%EC%9E%AC%EC%97%AD{quote("양재역")}'
req = requests.get(url)
html = req.text
#html

## 찾으려는 데이터의 태그 찾기

In [12]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

In [13]:
lis = soup.select('div.listTy1 > ul > li')
len(lis)

12

In [15]:
li = lis[0]
href = li.select_one('a')['href']

In [18]:
url = base_url + href
req = requests.get(url)
sub_soop = BeautifulSoup(req.text,'html.parser')

In [25]:
store = sub_soop.select_one('.title')
info = store.find('h3').get_text()
info

'그릭슈바인 양재역점3.6HOT'

In [20]:
score = store.select_one('h3 > strong').string
score

'3.6'

In [24]:
name = info[:info.find(score)]
name

'그릭슈바인 양재역점'

In [27]:
menu = sub_soop.select('.store_info p')[1].get_text()

'나이트라이프 > 맥주/호프'

In [28]:
tel = sub_soop.select_one('.p_tel p').get_text()
tel

'02-572-6100'

In [30]:
addr = sub_soop.select_one('.txt_adr').get_text()
addr

'서울특별시 서초구 강남대로 224'

In [33]:
name_list, score_list, menu_list,tel_list, addr_list = [],[],[],[],[]

for i, li in enumerate(lis):
    href = li.select_one('a')['href']
    url = base_url + href
    req = requests.get(url)
    sub_soop = BeautifulSoup(req.text,'html.parser')

    try:
        store = sub_soop.select_one('.title')
        info = store.find('h3').get_text()
        score = store.select_one('h3 > strong').string
        name = info[:info.find(score)]
        menu = sub_soop.select('.store_info p')[1].get_text()
        tel = sub_soop.select_one('.p_tel p').get_text()
        addr = sub_soop.select_one('.txt_adr').get_text()
    except:
        print(i)

    name_list.append(name)
    score_list.append(score)
    menu_list.append(menu)
    tel_list.append(tel)
    addr_list.append(addr)

3
4
